In [14]:
from uproot_io import Events, View
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

In [15]:
thr_std=0.2 #some constants
n_events=9310

In [13]:
print("Running...")
toc = time.perf_counter() #start timer


E = Events("CheatedRecoFile_1.root") #import data

view_u = View(E, "u") #unpack to view
view_v = View(E, "v")
view_w = View(E, "w")


u_x = view_u.x #unpack each plane
u_z = view_u.z
u_adc = view_u.adc
u_truevtx_z = view_u.true_vtx_z
u_truevtx_x = view_u.true_vtx_x


v_x = view_v.x #unpack each plane
v_z = view_v.z
v_adc = view_v.adc
v_truevtx_z = view_v.true_vtx_z
v_truevtx_x = view_v.true_vtx_x

w_x = view_w.x #unpack each plane
w_z = view_w.z
w_adc = view_w.adc
w_truevtx_z = view_w.true_vtx_z
w_truevtx_x = view_w.true_vtx_x

tic = time.perf_counter() #stop timer
print("All data loaded in", tic-toc, "seconds")

Running...
All data loaded in 214.7037135999999 seconds


In [16]:
def AoI(plane, event_number, thresholding=True, no_std=thr_std, ordering=True):
    #given plane and event number, extract required information and threshold if required
    if plane=="u": #unpack data from required plane
        AoI_x = u_x[event_number]
        AoI_z = u_z[event_number]
        AoI_adc = u_adc[event_number]
        AoI_truevtxz = u_truevtx_z[event_number]
        AoI_truevtxx = u_truevtx_x[event_number]
    elif plane=="v": #unpack data from required plane
        AoI_x = v_x[event_number]
        AoI_z = v_z[event_number]
        AoI_adc = v_adc[event_number]
        AoI_truevtxz = v_truevtx_z[event_number]
        AoI_truevtxx = v_truevtx_x[event_number]
    elif plane=="w": #unpack data from required plane
        AoI_x = w_x[event_number]
        AoI_z = w_z[event_number]
        AoI_adc = w_adc[event_number]   
        AoI_truevtxz = w_truevtx_z[event_number]
        AoI_truevtxx = w_truevtx_x[event_number]
    else:
        print("PLane not u,v,w")
        
    if thresholding: #remove data points with energies far away from the mean
        mean = np.average(AoI_adc)
        std = np.std(AoI_adc)
        AoI_x = AoI_x[(mean-no_std*std<AoI_adc)]
        AoI_z = AoI_z[(mean-no_std*std<AoI_adc)]
        AoI_adc = AoI_adc[(mean-no_std*std<AoI_adc)]
        
    if ordering: #order data along z such that AoI[0] is min_z and AoI[-1] is max_z
        order = np.argsort(AoI_z)
        AoI_z = np.take_along_axis(AoI_z, order, axis=0)
        AoI_x = np.take_along_axis(AoI_x, order, axis=0)
        AoI_adc = np.take_along_axis(AoI_adc, order, axis=0)
        
    AoI = [AoI_z, AoI_x, AoI_adc]
    true = (AoI_truevtxz,AoI_truevtxx)
    return AoI, true #easy to unpack form


In [17]:
Aoi, true = AoI('u', 20)

In [18]:
#least z stuff
def least_z(AoI):
    z, x, adc = AoI
    return z[0], x[0]

def least_z_scores(AoI, dependence=np.exp, plot=False):
    z, x, adc = AoI
    scores = dependence(z[0]-z)
    if plot:
        plt.plot(z, x, 'o')
        plt.show()
        plt.plot(z, scores, 'ro')
        plt.show()
    return scores

In [19]:
#rphi stuff

def rfunc(r):
    #functional r dependence - optimal at inv_sqrt(r)
    return 1/(r+0.01)**0.5


def rphi(AoI, centre_point, rdep=rfunc, no_sectors=12, centre=0):
    #take the point number to make a centre, divide into radial sectors and plot a histogram of where points lie
    
    z, x, adc = AoI #import data

    cen_x = x - x[centre_point] #centre data around centre_point
    cen_z = z - z[centre_point]

    theta = np.arctan2(cen_x, cen_z) #get all polar angles
    r2 = cen_x**2 + cen_z**2 #and squared polar radial coordinate
    
    rweight = rdep(r2) #get weights from functional form of radial co-ord
              
    hist_data, hist_bins = np.histogram(theta, no_sectors, (-np.pi+centre,np.pi+centre), weights=rweight) #turn into histogram data
    hist_data = (hist_data - np.roll(hist_data, int(len(hist_data)/2)))[x>0] #subract mirrored data
    

    score = np.sum(hist_data**2) # score for this point as centre
    norm_score = (score/len(theta)**2)**0.5 #normalise by size of event
    
    return norm_score



def rphi_scores(AoI, rdep=rfunc, no_sectors=12, centre=0):
    z, x, adc = AoI
    L = len(z)
    scores = np.zeros(L)
    for i in range(L):
        scores[i] = rphi(AoI, i, rdep, no_sectors, centre)
    return scores


In [ ]:
#quadvertex stuff

def line_finder(point1, point2):
    #given 2 points both (x,y) find y=mx+c and return (m,c)
    slope = (point1[1] - point2[1])/(point1[0] - point2[0])
    intercept = point1[1] - slope * point1[0]
    return slope, intercept

def line_intersect(slope1, intercept1, slope2, intercept2):
    #given 2 of these (m,c), find (xint, yint) where these lines cross
    x_point = (intercept2 - intercept1)/(slope1 - slope2)
    y_point = slope1 * x_point + intercept1
    return x_point, y_point

def select_random_points(AoI, no_iter):
    #randomly generate 4 points, draw lines through them and find their intersection. do this no_iter times
    z, x, adc = Aoi #unpack data
    
    vertex_length = len(z) #no points in event
    initial_vertex_points = np.zeros((no_iter, 2)) #(x,y) for every iteration
    
    if vertex_length > 4:
        range_x = (np.amin(AoI_x), np.amax(AoI_x)) #define exclusion rectangle
        range_z = (np.amin(AoI_z), np.amax(AoI_z))
        
        for i in range(no_iter):
            sample = random.sample(range(0, vertex_length), 4) #pick 4 indices corresponding to points w/i event
            vtxind_1, vtxind_2, vtxind_3, vtxind_4 = sample[0], sample[1], sample[2], sample[3]
            point1, point2, point3, point4 = (x[vtxind_1], z[vtxind_1]), (x[vtxind_2], z[vtxind_2]), (x[vtxind_3], z[vtxind_3]), (x[vtxind_4], z[vtxind_4])
            #could use numpy random to give multi-dimensional ranodm array here instead of all this unpacking?
            
            slope1, intercept1 = line_finder(point1, point2) #pick pairs and find their intercepts
            slope2, intercept2 = line_finder(point3, point4)
            x_point, y_point = line_intersect(slope1, intercept1, slope2, intercept2)
            #could permute (1,2)(3,4); (1,3)(2,4); (1,4)(2,3) to get 3 data points from 1 randomisation
            # marginal performance improvement? - demonstarted below
            '''
            points = [point1, point2, point3, point4]
            initial_vertex_points = np.zeros((no_iter, 3, 2)) #now 3 points at each iteration
            for j in range(3):
                slope1, intercept1 = line_finder(points[0], points[1+j]) #pick pairs and find their intercepts
                slope2, intercept2 = line_finder(points[(j+2)%3+1], points[(j+2)%3 + 3*(j==1)]) #this is silly but works
                x_point, y_point = line_intersect(slope1, intercept1, slope2, intercept2)
                initial_vertex_points[i,j] = x_point, y_point
            '''
            
            if x_point > range_x[0] and x_point < range_x[1] and y_point > range_z[0] and y_point < range_z[1]:
                #if inside exclusion rectangle, add intersect point to store
                initial_vertex_points[i, 0], initial_vertex_points[i, 1] = x_point, y_point
                #initial_vertex_points[i] = x_point, y_point

    return initial_vertex_points

def heatmap4points(AoI, no_iter=100, binwdith=100):
    #plot histogram of line intersections
    z, x, adc = AoI #unpack data
    
    vertex_points = select_random_points(AoI, no_iter) #import intersections data
    
    histvertex = plt.hist2d(vertex_points[:,1], vertex_points[:,0], bins=(binwdith, binwdith), cmap=plt.cm.jet) #2d hist data
    
    histogram_array = histvertex[0]
    
    max_value = np.amax(histogram_array)
    indices = np.where(histogram_array == max_value)
    #argmax accomplishes this more simply
    #indices = np.argmax(histogram_array)
    
    z_value = histvertex[1][indices[0]]
    x_value = histvertex[2][indices[1]]

    return z_value[0], x_value[0]

def quadvertex_scores(AoI, no_iter=100, binwdith=100):
    #plot histogram of line intersections
    z, x, adc = AoI #unpack data
    L = len(z)
    
    vertex_points = select_random_points(AoI, no_iter) #import intersections data
    
    histogram_array, zedges, xedges, image = plt.hist2d(vertex_points[:,1], vertex_points[:,0], bins=(binwdith, binwdith)) #2d hist data
    
    zbins = np.digitize(z, zedges)
    xbins = np.digitize(x, xedges)
    
    scores = np.zeros(L)
    
    for i in range(L):
        scores[i] = histogram_array[zbins[i], xbins[i]]

    return scores

In [12]:
def pdf(plane, event, limit, algorithm_scores, add_args=()):
    #leastz add_args = (dependence=np.exp, plot=False)
    #rphi add_args = (rdep=rfunc, no_sectors=12, centre=0)
    #quadvertex add_args = (no_iter=100, binwdith=100)
    
    Aoi, true = AoI(plane, event)
    z, x, adc = Aoi
    ztrue, xtrue = true
    L = len(z)
    
    scores = algorithm_scores(Aoi, *add_args)
    
    r = ( (z-ztrue)**2 + (x-xtrue)**2 )**0.5 #distances from true vertex
    truth = r<limit
    
    signal = [i for (i, v) in zip(scores, truth) if v]
    noise = [i for (i, v) in zip(scores, truth) if not v]
    
    return signal, noise



pdf('u', 300, 4, least_z_scores, plot=True)


([6.1160695e-32,
  3.8352017e-32,
  2.4049385e-32,
  1.5080523e-32,
  9.456538e-33,
  5.929908e-33,
  3.718437e-33,
  3.718437e-33,
  2.3317189e-33,
  1.46215e-33,
  9.168629e-34,
  5.7493686e-34,
  5.7493686e-34,
  3.6052545e-34,
  3.6052545e-34,
  2.2607456e-34,
  2.2607456e-34,
  1.4176341e-34,
  1.4176341e-34,
  8.8895526e-35,
  5.5743687e-35],
 [1.0,
  1.3663183e-05,
  8.567833e-06,
  8.863108e-20,
  2.118876e-21,
  1.7601463e-27,
  1.1037343e-27,
  9.168629e-34,
  5.7493686e-34,
  5.7493686e-34,
  3.6052545e-34,
  8.8895526e-35,
  5.5743687e-35,
  3.4954906e-35,
  3.4954906e-35,
  2.191916e-35,
  2.191916e-35,
  1.374484e-35,
  1.374484e-35,
  8.6189055e-36,
  5.404654e-36,
  5.404654e-36,
  3.3890943e-36,
  2.1251982e-36,
  1.3326371e-36,
  8.356562e-37,
  5.2401065e-37,
  3.2859366e-37,
  2.0604953e-37,
  1.2920642e-37,
  8.102204e-38,
  5.080607e-38,
  3.1858947e-38,
  1.9977777e-38,
  1.2527362e-38,
  7.855528e-39,
  4.925963e-39,
  3.088898e-39,
  1.936955e-39,
  1.214605e-3